<a href="https://colab.research.google.com/github/vitorjexs/2024-05-imersao-alura-google-gemini/blob/main/Google_Gemini_Imers%C3%A3o_Alura_2024_05_Aula_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get("SECRET_KEY")

genai.configure(api_key=api_key)

In [6]:
for model in genai.list_models():
  if "embedContent" in model.supported_generation_methods:
    print(model.name)

models/embedding-001
models/text-embedding-004


Exemplo de Embed

In [10]:
title = "Por que é tão difícil criar exemplos próprios?"
lorenIpsum = ("É só um exercício de exemplo"
              "/n"
              "Acabei de pular uma linha"
              "/n"
              "De novo")

embedding = genai.embed_content(model="models/embedding-001",
                                content=lorenIpsum,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")

print(embedding)


{'embedding': [0.008746391, -0.034893136, -0.03394119, 0.014982169, 0.036144633, -0.011821693, 0.015729787, -0.020466588, 0.020250805, 0.07652906, 0.03751925, -0.03155111, 0.0252817, -0.044768777, 0.019831758, -0.04377015, 0.016623242, 0.0036166261, -0.021051358, -0.037695207, -0.0053620883, -0.0047493377, 0.0043503535, 0.014656799, 0.006340597, -0.0061414274, 0.034373097, -0.057881556, -0.029245043, 0.01239787, -0.0773617, 0.061385103, -0.055571437, -0.009894369, -0.041354354, -0.05260357, -0.021316784, -0.0011463301, 0.01889996, 0.013386572, 0.011048815, -0.043511808, -0.05155436, 0.0367325, 0.029038595, -0.0012751918, 0.030055746, -0.0026345286, 0.009485613, -0.08618699, 0.0009294035, 0.016807484, 0.10526586, -0.019681582, -0.04650135, -0.059758876, 0.06279923, -0.006708936, 0.019021507, 0.0007351958, -0.037509225, 0.009639039, 0.0375129, 0.0032542683, -0.009712607, -0.082444154, -0.054573093, 0.022349488, 0.04989662, 0.048817746, 0.010164624, -0.023226395, 0.012540821, 0.032569796,

Documentos a serem buscados:

In [11]:
DOCUMENT1 = {
    "Title":"As cores do Arco-íris",
    "Content":"São 7, mas esqueci"
}

DOCUMENT2 = {
    "Title":"A teoria das cores no cinema",
    "Content":"Elas mexem com as emoções"
}

DOCUMENT3 = {
    "Title":"O poder das cores",
    "Content":"Os Super Saiyajins são amarelos, a cor clássica da alegria"
}

allDocuments = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [17]:
df = pd.DataFrame(allDocuments)
df.columns = ["Title", "Content"]
df

,Title,Content
0,As cores do Arco-íris,"São 7, mas esqueci"
1,A teoria das cores no cinema,Elas mexem com as emoções
2,O poder das cores,"Os Super Saiyajins são amarelos, a cor clássic..."


In [18]:
model = "models/embedding-001"

In [24]:
def embed_fn (title,content):
  return genai.embed_content(model=model,
                             content=content,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [27]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Title"], row["Content"]), axis=1)
df

,Title,Content,Embeddings
0,As cores do Arco-íris,"São 7, mas esqueci","[0.048127953, -0.019322164, -0.07232691, -0.02..."
1,A teoria das cores no cinema,Elas mexem com as emoções,"[0.024589295, 0.012364421, -0.049210183, 0.014..."
2,O poder das cores,"Os Super Saiyajins são amarelos, a cor clássic...","[-0.008658696, -0.031141583, -0.06156194, 0.00..."


In [33]:
def generate_and_search(consult,base,model):
  searchResultEmbedding = genai.embed_content(model=model,
                             content=consult,
                             task_type="RETRIEVAL_QUERY")["embedding"]

  scaledProducts = np.dot(np.stack(df["Embeddings"]), searchResultEmbedding)

  index = np.argmax(scaledProducts)

  return df.iloc[index]["Content"]

In [34]:
search = "Qual a cor dos Saiyajins"

generate_and_search(search, df, model)

'Os Super Saiyajins são amarelos, a cor clássica da alegria'